# Packages
Se usa plotly para graficar
 - Instalar paquete: conda install jupyterlab "ipywidgets=7.5"

In [155]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# Load of files

In [156]:
pwd()

'C:\\Users\\rbonilla\\Documents\\GitHub\\ds4a_team80\\0_eda'

In [157]:
dataBiologicos = pd.read_excel(r'C:\Users\rbonilla\Google Drive\work\TheProfessional\Cursos\DS4A\team80\codigo\data\Biologicos Asma.xlsx', sheet_name='data')
dataBiologicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12594 entries, 0 to 12593
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Identificacion        12594 non-null  int64         
 1   Fecha_Dcto            12594 non-null  datetime64[ns]
 2   Nombre_Producto_Mvto  12594 non-null  object        
 3   Nom_Generico          12594 non-null  object        
 4   Cantidad              12594 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 492.1+ KB


In [158]:
dataBiologicos = dataBiologicos.loc[dataBiologicos['Cantidad']>=0]
dataBiologicos.head()

,Identificacion,Fecha_Dcto,Nombre_Producto_Mvto,Nom_Generico,Cantidad
0,1853167,2019-08-02,OMALIZUMAB 150 MG POLV INY (XOLAIR)-21755-4896,OMALIZUMAB,2
1,695889,2019-07-31,OMALIZUMAB 150 MG POLV INY (XOLAIR)-21755-4896,OMALIZUMAB,2
2,1534187,2019-01-05,OMALIZUMAB SOLUCIÓN INYECTABLE 150MG /1ML (XOL...,OMALIZUMAB,4
3,1831103,2018-12-03,OMALIZUMAB SOLUCIÓN INYECTABLE 150MG /1ML (XOL...,OMALIZUMAB,3
4,1831103,2019-01-03,OMALIZUMAB SOLUCIÓN INYECTABLE 150MG /1ML (XOL...,OMALIZUMAB,3


# Counts by variable

Se crea una columna igual al indice para aplicar las funciones de agrupación por fecha (año o mees, etc.)

In [159]:
dataBiologicos.index = dataBiologicos.Fecha_Dcto
#dataBiologicos["FullDate"] = dataBiologicos.index
#dataBiologicos['Year'] = dataBiologicos['Fecha_Dcto'].dt.year
dataBiologicos['Year'] = dataBiologicos.Fecha_Dcto.dt.year
dataBiologicos['Month'] = dataBiologicos.Fecha_Dcto.dt.month

Se crea un df temporal para la gráfica
Los años se vuelven categorías para agrupar facilmente

In [160]:
dftemp = dataBiologicos.pivot_table(values = 'Cantidad',
    index=['Nom_Generico', 'Year'],
    columns=None,
    aggfunc='sum').reset_index()

dftemp['Year'] = dftemp.Year.astype('category')
dftemp = dftemp.sort_values(by = ['Year'])

## Figure

In [161]:
fig = px.bar(data_frame=dftemp, x = 'Nom_Generico', y = "Cantidad",
             color = "Year",
             color_discrete_sequence = px.colors.qualitative.Vivid,
             barmode='group',
            text = "Cantidad")

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_layout(yaxis_type='log',
                  title_text='Delivered MK by year',
                  xaxis_title="MK",
                  yaxis_title="Total count",
                  font=dict(
                      #family="Courier New, monospace",
                      size=12,
                      #color="#7f7f7f"
                  ),
                  plot_bgcolor = '#BABABA'
                 )

fig.update_traces(
    #marker_color='rgb(158,202,225)',
    marker_line_color='#303030',
    marker_line_width=1.5,
    opacity=0.9)

fig.show()

In [162]:
dftemp = dataBiologicos.groupby(['Nom_Generico',pd.Grouper(key='Fecha_Dcto', freq='M')])['Cantidad'].sum()
dftemp = dftemp.reset_index()
dftemp.head()

,Nom_Generico,Fecha_Dcto,Cantidad
0,BENRALIZUMAB,2019-03-31,1
1,BENRALIZUMAB,2019-04-30,5
2,BENRALIZUMAB,2019-05-31,12
3,BENRALIZUMAB,2019-06-30,21
4,BENRALIZUMAB,2019-07-31,18


In [163]:
fig = px.line(data_frame=dftemp[dftemp['Fecha_Dcto']>'2018-01-01'], x='Fecha_Dcto', y = 'Cantidad',
              color = 'Nom_Generico')

fig.update_layout(yaxis_type='log')
fig.update_layout(title_text = "Monthly evolution of MK claims",
                 xaxis_title = "Month",
                 yaxis_title = "Count",
                  legend_title_text=None,
                  font=dict(
                      #family="Courier New, monospace",
                      size=12,
                      #color="#7f7f7f"
                  )
                 )
fig.show()

# Time in Treatment

Time is calculate from the first medication clain until de last in the database

In [164]:
dataBiologicos.columns

Index(['Identificacion', 'Fecha_Dcto', 'Nombre_Producto_Mvto', 'Nom_Generico',
       'Cantidad', 'Year', 'Month'],
      dtype='object')

In [165]:
dftemp = dataBiologicos.groupby('Identificacion').agg({'Fecha_Dcto':['min', 'max', lambda x: (x.max() - x.min()).days]})
dftemp.columns = dftemp.columns.droplevel(0)
dftemp.head()

grep = dftemp.columns.str.contains(pat = 'lambda')
colname = dftemp.columns[grep]
colname[0]
dftemp.rename(columns = {colname[0]:"Delta_Time"}, inplace = True)

dftemp.head()

,min,max,Delta_Time
Identificacion,,,
500547,2019-07-13,2019-07-13,0
500588,2017-12-07,2019-10-19,681
502989,2018-01-25,2020-04-22,818
506808,2018-01-22,2018-08-21,211
512938,2017-12-20,2020-05-20,882


In [166]:
dftemp.sort_values(by = "Delta_Time", ascending=False )

,min,max,Delta_Time
Identificacion,,,
1194113,2017-03-06,2020-05-20,1171
1671179,2016-11-16,2019-07-10,966
1715065,2017-12-07,2020-05-30,905
1561697,2017-10-17,2020-04-08,904
1729099,2017-12-06,2020-05-28,904
...,...,...,...
1534187,2019-01-05,2019-01-05,0
607756,2020-04-02,2020-04-02,0
1559689,2017-12-28,2017-12-28,0


## Figure

In [167]:
fig = px.histogram(data_frame=dftemp[(dftemp['Delta_Time']>=0) & (dftemp['Delta_Time'] < 1200)],
             x = 'Delta_Time',
             nbins = 50,
             marginal = "box",
            color_discrete_sequence=['#FF6E00'] # color of histogram bars
                  )
fig.update_layout(title_text = "Distribution of the ellapced time since first treatment to last one",
                 xaxis_title = "Time(days)",
                 yaxis_title = "No. of patients"
                 )

## Analysis
What we are seen here is that the proportion (or probability) of patients that are in the proyect (ashma) are equal.  
The probability that a patient has been in the study 1 day or 10 or 100 are the same.  
The number of patients with more that 900 days in the study are only 110. 

We face the problem that we have not enought data to perform a tiem seires analysis, at least in the traditional sense.

# Analysis of the amount of treatment (medication) has been delivered by Omnivida by patient.

In [168]:
dataBiologicos.columns

Index(['Identificacion', 'Fecha_Dcto', 'Nombre_Producto_Mvto', 'Nom_Generico',
       'Cantidad', 'Year', 'Month'],
      dtype='object')

In [169]:
#dftemp = dataBiologicos.groupby('Identificacion').agg({'Cantidad':sum, 'Month':'count'})
dftemp = dataBiologicos.groupby('Identificacion').agg(
    Cantidad_Total=pd.NamedAgg(column='Cantidad', aggfunc=sum),
    Conteo_Evento=pd.NamedAgg(column='Cantidad', aggfunc='count'),
    Delta_Time=pd.NamedAgg(
        column="Fecha_Dcto", 
        aggfunc=lambda x: (max(x) - min(x)).days)    
)
dftemp.head()
#px.bar(data_frame=dftemp, x = Delta_Time)

,Cantidad_Total,Conteo_Evento,Delta_Time
Identificacion,,,
500547,2,1,0
500588,72,28,681
502989,42,23,818
506808,12,9,211
512938,80,20,882


## Figure

In [170]:
fig = px.scatter(data_frame=dftemp, x='Delta_Time', y='Cantidad_Total',
                 marginal_x = "histogram",
                 marginal_y = "histogram"
                )
fig.update_layout(#xaxis_type = 'log',
                 #yaxis_type = 'log',
                 title_text = "Relation between Medication quantity and time in study",
                 xaxis_title = "Time (days) in the treatment",
                 yaxis_title = "No. of medication received"
                 )

## Analysis
We see that there is no simple relation between the amount of time in treatment and the amounth of medication received

# Analysis of change of medication

In [171]:
dftemp = dataBiologicos.groupby('Identificacion').agg(
    Cantidad_Total=pd.NamedAgg(column='Cantidad', aggfunc=sum),
    Conteo_Evento=pd.NamedAgg(column='Cantidad', aggfunc='count'),
    Delta_Time=pd.NamedAgg(
        column="Fecha_Dcto", 
        aggfunc=lambda x: (max(x) - min(x)).days),
    Different_MK = pd.NamedAgg(column = 'Nom_Generico', aggfunc = 'nunique')
).reset_index()
dftemp.head()

,Identificacion,Cantidad_Total,Conteo_Evento,Delta_Time,Different_MK
0,500547,2,1,0,1
1,500588,72,28,681,1
2,502989,42,23,818,1
3,506808,12,9,211,1
4,512938,80,20,882,1


## Figure

In [172]:
dftemp['Different_MK']=dftemp['Different_MK'].astype(str)
dftemp.head()
#dftemp = dftemp.reset_index()
dftemp2 = dftemp.groupby(['Different_MK']).count().reset_index()
#dftemp2 = dftemp2.reset_index()
fig = px.bar(data_frame=dftemp2, x='Different_MK', y = 'Identificacion')
fig = fig.update_layout(
    xaxis_title = "No. of differente treatment",
    yaxis_title = "No. of patients"
                 )
fig.update_traces(
    marker_line_color='#303030',
    marker_line_width=1.5,
    opacity=0.9
)

In [173]:
fig = px.scatter(data_frame=dftemp, x='Delta_Time', y='Cantidad_Total',
                 color = 'Different_MK',
                 #color_discrete_sequence = ['red','blue','green'],
                 color_discrete_sequence=px.colors.qualitative.G10,
                 marginal_x = "histogram",
                 marginal_y = "histogram",
                 height  = 800
                )
fig = fig.update_layout(#xaxis_type = 'log',
                 #yaxis_type = 'log',
                 title_text = "Relation between Medication quantity and time in study",
                 xaxis_title = "Time (days) in the treatment",
                 yaxis_title = "No. of medication received"
                 )
fig.update_traces(marker=dict(size=8,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

# Distribution of the time between medication claim

In [174]:
# We define an arbitrary initial time to perform differences
date_of_reference = datetime.strptime("2000-01-01", "%Y-%m-%d")

dftemp = dataBiologicos.groupby('Identificacion').agg(
    Delta_Time=pd.NamedAgg(
        column="Fecha_Dcto", 
        aggfunc=lambda x: [np.diff((x.sort_values() - date_of_reference).dt.days,axis = 0)])
    
    #lambda calculate the time interval between claims of medication
    # we use a date_of_reference as an arbitrary date to make calculation but is irrelevant
    
).reset_index()

# Flatten function
delta_claim = dftemp.Delta_Time.explode().explode()

# histogram
dftemp = delta_claim.value_counts().reset_index().sort_values(by='index')

## Figure

In [175]:
# Filter 0 values meanning two medication claim the same day
fig = px.bar(data_frame=dftemp[(0 < dftemp['index']) & (dftemp['index'] < 100)], x='index', y='Delta_Time')
fig.update_layout(
    xaxis = dict(#type = 'log',
                 title = "Time interval between MK claim"),
    yaxis = dict(#type = 'log',
                 title = "Count of events")
)

## Analysis
We observe a clear bimodal distribution with modes in 15 days and 30 days showing two kind of medication.
Of course the problem is we have deviation from the modes.